In [1]:
%pip install torch --index-url https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install "trl>=0.20.0" "peft>=0.17.0" "transformers>=4.55.0" trackio


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en", split="train")
dataset = dataset.shuffle(seed=42).select(range(700))
tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b", trust_remote_code=True)

def format_messages(example):
    messages = [
        {"role": "user",      "content": example["Question"]},
        {"role": "assistant", "content": example["Complex_CoT"]},
        {"role": "assistant", "content": example["Response"]},
    ]
    return {"text": tokenizer.apply_chat_template(messages, tokenize=False)}

dataset = dataset.map(format_messages, remove_columns=dataset.column_names)



In [3]:
dataset[0]

{'text': "<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-08-07\n\nReasoning: medium\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>user<|message|>In the instrument formula for a Gingival Margin Trimmer (GMT) used during cavity preparation, what is the second number representing the angle of the cutting edge when access to the distal gingival margin is achieved?<|end|><|start|>assistant<|channel|>final<|message|>Alright, so a Gingival Margin Trimmer, or GMT for short, is some sort of dental tool used during cavity prep. I need to figure out what that second number in its formula really means, especially when working with the distal gingival margin. Let's start with the basics about these numbers.\n\nThe first number in any dental instrument formula usually tells us the blade width, measured in tenths of a millimeter. Pretty straightforward.\n

In [4]:
import torch
from transformers import AutoModelForCausalLM, Mxfp4Config

quantization_config = Mxfp4Config(dequantize=True)
model_kwargs = dict(
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    use_cache=False,
    device_map="auto",
)

model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b", **model_kwargs)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules="all-linear",  # Enable injection into all linear layers
    target_parameters=[
        "5.mlp.experts.gate_up_proj",
        "5.mlp.experts.down_proj",
        "7.mlp.experts.gate_up_proj",
        "7.mlp.experts.down_proj",
        "15.mlp.experts.gate_up_proj",
        "15.mlp.experts.down_proj",
    ],
)

peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()


trainable params: 15,040,512 || all params: 20,929,797,696 || trainable%: 0.0719


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/peft/tuners/lora/layer.py:159: UserWarning: Unsupported layer type '<class 'transformers.models.gpt_oss.modeling_gpt_oss.GptOssExperts'>' encountered, proceed at your own risk.
  warnings.warn(


In [8]:
from trl import SFTTrainer, SFTConfig

training_args = SFTConfig(
    num_train_epochs=1,
    learning_rate=2e-4,
    logging_steps=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    save_steps=100,
    save_total_limit=2,
    gradient_checkpointing=True,
    bf16=True,  
    push_to_hub=True,
    output_dir="gpt-oss-medical",
    remove_unused_columns=False,
    max_length=2048
)


In [9]:
trainer = SFTTrainer(
    model=peft_model,
    args=training_args,
    processing_class=tokenizer,
    train_dataset=dataset
)

trainer.train()


Step,Training Loss
1,1.991100
2,1.782700
3,1.737200
4,1.577700
5,1.497200
6,1.366600
7,1.474400
8,1.448200
9,1.309200
10,1.390400


TrainOutput(global_step=44, training_loss=1.3502155867489902, metrics={'train_runtime': 505.9826, 'train_samples_per_second': 1.383, 'train_steps_per_second': 0.087, 'total_flos': 6.912990403511962e+16, 'train_loss': 1.3502155867489902})

In [10]:
trainer.save_model(training_args.output_dir)
trainer.push_to_hub(dataset_name="gpt-oss-med")

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...ts.1754568242.192-222-54-41.17628.0: 100%|##########| 7.36kB / 7.36kB            

  ...o/gpt-oss-medical/training_args.bin: 100%|##########| 6.16kB / 6.16kB            

  ...ts.1754568302.192-222-54-41.17628.1: 100%|##########| 21.0kB / 21.0kB            

...

  ...ts.1754568302.192-222-54-41.17628.1: 100%|##########| 21.0kB / 21.0kB            

  ...ts.1754568242.192-222-54-41.17628.0: 100%|##########| 7.36kB / 7.36kB            

  ...s-medical/adapter_model.safetensors: 100%|##########| 60.2MB / 60.2MB            

  ...udio/gpt-oss-medical/tokenizer.json: 100%|##########| 27.9MB / 27.9MB            

CommitInfo(commit_url='https://huggingface.co/Sandipan1976/gpt-oss-medical/commit/90ca1988ca80df8f53c091877be72240ee748aab', commit_message='End of training', commit_description='', oid='90ca1988ca80df8f53c091877be72240ee748aab', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Sandipan1976/gpt-oss-medical', endpoint='https://huggingface.co', repo_type='model', repo_id='Sandipan1976/gpt-oss-medical'), pr_revision=None, pr_num=None)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")

# Load the original model first
model_kwargs = dict(attn_implementation="eager", torch_dtype="auto", use_cache=True, device_map="auto")
base_model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b", **model_kwargs).cuda()

# Merge fine-tuned weights with the base model
peft_model_id = "gpt-oss-medical"
model = PeftModel.from_pretrained(base_model, peft_model_id)
model = model.merge_and_unload()

MXFP4 quantization requires triton >= 3.4.0 and triton_kernels installed, we will default to dequantizing the model to bf16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/peft/tuners/lora/layer.py:159: UserWarning: Unsupported layer type '<class 'transformers.models.gpt_oss.modeling_gpt_oss.GptOssExperts'>' encountered, proceed at your own risk.
  warnings.warn(


In [4]:
USER_PROMPT = "What medicines should I take for spondilytis or leg pain?" 
messages = [
    
    {"role": "user", "content": USER_PROMPT},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to(model.device)

gen_kwargs = {"max_new_tokens": 1025, "do_sample": True, "temperature": 0.6, "top_p": None, "top_k": None}

output_ids = model.generate(input_ids, **gen_kwargs)
response = tokenizer.batch_decode(output_ids)[0]
print(response)

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-07

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>user<|message|>What medicines should I take for spondilytis or leg pain?<|end|><|start|>assistant<|channel|>final<|message|>I’m not a medical professional, but I can share some general information about common treatments for spondylitis and leg pain. It’s always best to consult a healthcare provider for personalized advice.

Spondylitis, which is an inflammation of the vertebrae, can be treated with various medications depending on the type and severity. Here are a few options that are commonly considered:

1. **Nonsteroidal Anti-Inflammatory Drugs (NSAIDs)**: Medications like ibuprofen or naproxen can help reduce pain and inflammation. They’re often used as a first-line treatment.

2. **Corticosteroids**: For more severe inflam